In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

# Ensure version is 2.x
print(tf.__version__)

# If using tf.compat.v1 to enable eager execution, it should be done at the very start
tf.compat.v1.enable_eager_execution()

# Now other TensorFlow operations can follow
tensor = tf.constant([1, 2, 3])
print(tensor.numpy())

2.16.1

[1 2 3]


In [2]:
tf.executing_eagerly()

True

In [3]:
print(tf.__version__)

2.16.1


### Creating tensors

In [4]:
string = tf.Variable('string', tf.string)
tf.rank(string) # X ~ (1)

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [5]:
number = tf.Variable(123)
tf.shape(number)

<tf.Tensor: shape=(0,), dtype=int32, numpy=array([], dtype=int32)>

In [6]:
rank1_tensor = tf.Variable([1,2,3,4,5,6], tf.int64)
tf.rank(rank1_tensor)


<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [7]:
rank2_tensor = tf.Variable([['a', 'b'], ['d', 'e']], tf.string) # numpy=2 means rank
tf.rank(rank2_tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [8]:
rank1_tensor.shape

TensorShape([6])

In [9]:
rank2_tensor.shape

TensorShape([2, 2])

### Changing a shape
Shape of a tensor is the amount of elements that exist in each dimension 

In [10]:
tensor1 = tf.ones([1,2,3]) # matrix of 1s dimension 2x3
print(tensor1)

tf.Tensor(
[[[1. 1. 1.]
  [1. 1. 1.]]], shape=(1, 2, 3), dtype=float32)


In [11]:
tensor2 = tf.ones([1,5,4]) # matrix of 1s dimension 5x4
print(tensor2)

tf.Tensor(
[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]], shape=(1, 5, 4), dtype=float32)


In [12]:
tensor1_reshaped = tf.reshape(tensor1, [2,1,3])

print(tensor1_reshaped)

tf.Tensor(
[[[1. 1. 1.]]

 [[1. 1. 1.]]], shape=(2, 1, 3), dtype=float32)


In [13]:
tensor2_reshaped = tf.reshape(tensor2, [5, -1]) # -1 tells to calculate the size of the dimetsion in that place
print(tensor2_reshaped)

tf.Tensor(
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]], shape=(5, 4), dtype=float32)


### Types of tensors

Variable - the only changeable one (like reshaping above)

Constant

Placeholder

Sparse Tensor

### Evaluating a tensor
Evaluating a tensor means to get its value. By creating a session.

In [14]:
# with tf.Session() as sess:
#     tensor1.eval

In [15]:
# With Tensorflow 2.x, eager mode is enabled by default. so the following code works with TF2.0.

# import tensorflow as tf
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])
product = tf.matmul(matrix1, matrix2)

#print the product
print(product)         # tf.Tensor([[12.]], shape=(1, 1), dtype=float32)
print(product.numpy()) # [[12.]]

tf.Tensor([[12.]], shape=(1, 1), dtype=float32)
[[12.]]


Multiplication

In [16]:
tensor_zeros = tf.zeros([3,3], tf.int32)
print(tensor_zeros)
print(tensor_zeros.shape)

tf.Tensor(
[[0 0 0]
 [0 0 0]
 [0 0 0]], shape=(3, 3), dtype=int32)
(3, 3)


In [17]:
tensor_zeros_2 = tf.zeros([3,3], tf.int32)

In [18]:
multiplication = tf.multiply(tensor_zeros, tensor_zeros_2)
print(multiplication.shape)

(3, 3)


In [19]:
print(tf.reshape(tensor_zeros, [9]))

tf.Tensor([0 0 0 0 0 0 0 0 0], shape=(9,), dtype=int32)


In [20]:
a = tf.constant([1, 2])
b = tf.constant([3, 4])
adder_node = a + b

# Directly evaluate tensors in eager execution
print(adder_node.numpy())

[4 6]


In [21]:
df = pd.read_csv('insurance.csv')

In [22]:
train, test =  train_test_split(df, test_size=.2, random_state=37)

In [23]:
categorical_vars = df.select_dtypes(include=['object', 'category']).columns.to_list()
numeric_vars = df.select_dtypes(include=['object', 'number']).columns.to_list()

In [24]:
feature_vars = []

for i in categorical_vars:
    vocabulary = train[i].unique()
    feature_vars.append(tf.feature_column.categorical_column_with_vocabulary_list(i, vocabulary))

for i in numeric_vars:
    feature_vars.append(tf.feature_column.numeric_column(i))

    print(feature_vars)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='smoker', vocabulary_list=('no', 'yes'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='region', vocabulary_list=('northwest', 'southwest', 'southeast', 'northeast'), dtype=tf.string, default_value=-1, num_oov_buckets=0), NumericColumn(key='age', sha

### Input function

Input function converts pandas df into an object. Because TensorFlow model requires input data to have tf.data.Dataset object format.

In [25]:
def make_input_fun(df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(df), label_df)) # create tf.data.Dataset 
        if shuffle: 
            ds = ds.shuffle(1000) # shuffle a ds
            ds = ds.batch(batch_size).repeat(num_epochs) # splits ds into 32 blocks. repeats 10 epochs times
            return ds # returns a batch of a dataset
        return input_function  # returns an object to use in a TensorFlow model
    

train_input_fun = make_input_fun(df, train)
eval_input_fun = make_input_fun(df, test, num_epochs=1, shuffle=False)


In [26]:
train_input_fun = make_input_fun(df, train)
eval_input_fun = make_input_fun(df, test, num_epochs=1, shuffle=False)

fit = tf.estimator.LinearClassifier(feature_vars)

AttributeError: module 'tensorflow' has no attribute 'estimator'

#### Linear regression

In [ ]:
fit = tf1.est

In [ ]:
tf1.__version__

'2.16.1'

In [ ]:
print(dir(tf))  # This will list all available attributes and help identify if 'estimator' is truly missing.

['AggregationMethod', 'Assert', 'CriticalSection', 'DType', 'DeviceSpec', 'GradientTape', 'Graph', 'IndexedSlices', 'IndexedSlicesSpec', 'Module', 'Operation', 'OptionalSpec', 'RaggedTensor', 'RaggedTensorSpec', 'RegisterGradient', 'SparseTensor', 'SparseTensorSpec', 'Tensor', 'TensorArray', 'TensorArraySpec', 'TensorShape', 'TensorSpec', 'TypeSpec', 'UnconnectedGradients', 'Variable', 'VariableAggregation', 'VariableSynchronization', '_API_MODULE', '_KerasLazyLoader', '__all__', '__builtins__', '__cached__', '__compiler_version__', '__cxx11_abi_flag__', '__cxx_version__', '__doc__', '__file__', '__git_version__', '__internal__', '__loader__', '__monolithic_build__', '__name__', '__operators__', '__package__', '__path__', '__spec__', '__version__', '_api', '_compat', '_current_file_location', '_current_module', '_distutils', '_fi', '_initializers', '_inspect', '_kernel_dir', '_ll', '_losses', '_major_api_version', '_metrics', '_module_dir', '_module_util', '_names_with_underscore', '_o

In [ ]:
item = 'estimate'
my_list = dir(tf)

dir(tf).

['AggregationMethod',
 'Assert',
 'CriticalSection',
 'DType',
 'DeviceSpec',
 'GradientTape',
 'Graph',
 'IndexedSlices',
 'IndexedSlicesSpec',
 'Module',
 'Operation',
 'OptionalSpec',
 'RaggedTensor',
 'RaggedTensorSpec',
 'RegisterGradient',
 'SparseTensor',
 'SparseTensorSpec',
 'Tensor',
 'TensorArray',
 'TensorArraySpec',
 'TensorShape',
 'TensorSpec',
 'TypeSpec',
 'UnconnectedGradients',
 'Variable',
 'VariableAggregation',
 'VariableSynchronization',
 '_API_MODULE',
 '_KerasLazyLoader',
 '__all__',
 '__builtins__',
 '__cached__',
 '__compiler_version__',
 '__cxx11_abi_flag__',
 '__cxx_version__',
 '__doc__',
 '__file__',
 '__git_version__',
 '__internal__',
 '__loader__',
 '__monolithic_build__',
 '__name__',
 '__operators__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_api',
 '_compat',
 '_current_file_location',
 '_current_module',
 '_distutils',
 '_fi',
 '_initializers',
 '_inspect',
 '_kernel_dir',
 '_ll',
 '_losses',
 '_major_api_version',
 '_metrics',
 

In [ ]:

if item in my_list:
    print("Desired item is in list")